# Задание 1
Необходимо составить регулярку для поиска времени в тексте. Время имеет формат часы:минуты. И часы, и минуты состоят из двух цифр, пример: 09:00. Напишите регулярное выражение для поиска времени в строке: «Завтрак в 09:00». Учтите, что «37:98» – некорректное время.

In [1]:
import re
 
string = "Завтрак в 03:45 fdd"
pattern = '\s(2[0-3]|[0-1]\d:[0-5]\d)\s'
 
a = re.search(pattern, string)
if a:
    print(a.group())
else:
    print('Nothing found')

 03:45 


# Задание 2
Токенизировать текст

WordBrokers = "\;\:\-\—\<\>\\\"\.\,\!\?\&\=\+\(\)\{\}\[\]\r\n\t\« \»\“\”\„\/"

In [2]:
text = """\ Mr. Smith bought cheapsite.com for 1.5 million dollars, ie he paid a lot for it. Did he mind?
Adam Jones Jr. thinks he didn't. In any case, this isn't true... Well, with a probability of .9 it isn't. """ 
def tokinization(text):
    
    doc = re.split(r' *[\;\:\-\—\<\>\\\"\.\,\!\?\&\=\+\(\)\{\}\[\]\r\n\t\« \»\“\”\„\/\d ]*\s', text) 
    tokens = set()
    for token in doc: 
        tokens.add(token.lower())
    tokens.discard('')
    tokens_array = []
    for token in tokens:
        tokens_array.append(token)
        
    return tokens_array

# Задание 3
Реализовать три алгоритма, имеряющих расстояние между словами

#### Расстояние Хэмминга
Расстояние Хэмминга  — это количество различающихся позиций для строк.

In [3]:
def hemming_distance(s1,s2):
    dist = 0
    max_len = max(len(s1),len(s2))
    min_len = min(len(s1),len(s2))
    for i in range(min_len):
        if s1[i] != s2[i]:
            dist += 1
    dist += (max_len - min_len)
    return dist/max_len

In [4]:
s1 = "wqeew"
s2 = "weqew"
hemming_distance(s1,s1)

0.0

#### Расстояние Джаро
Расстояние Джаро - dj между двумя заданными строками s1 и s2 это: 
* dj = (m/a + m/b + (m-t)/m)/3

Где:
* a — длина строки s1
* b - длина строки s2
* m — число совпадающих символов (см. ниже)
* t — половина числа транспозиций (см. ниже)

Два символа из s1 и s2 соответственно, считаются совпадающими только: если они одинаковы, и их позиции относительно друг-друга находятся не дальше, чем на d - максимальное расстояние для поиска,

Где:
* d = int(max(a,b)/2)-1

Каждый символ строки s1 сравнивается со всеми символами в s2 на допустимом расстоянии d. Количество совпадающих (но отличающихся порядковыми номерами) символов, которое делится на 2, определяет число транспозиций - tr. 

Таким образом:
* t = int(tr/2);

In [5]:
def jaro_distance(str1,str2):
    if len(str1) > len(str2):
        s1 = str2
        s2 = str1
    else:
        s1 = str1
        s2 = str2
    dist = 0
    max_len = max(len(s1),len(s2))
    min_len = min(len(s1),len(s2))
    a = len(s1)
    b = len(s2)
    d = int(max_len/2)-1
    e = 0
    m = 0
    for i in range(min_len):
        if s1[i] == s2[i]:
            e += 1
            m += 1
            continue 
        j = i-d
        if j<0 : 
            j=0
        if s2.find(s1[i],j,i+d) != -1:
            m += 1
    t = int((m - e + 1)/2)
    if m==0:
        return  1
    else:
        return 1-(m/a+m/b +(m-t)/m)/3
    
    

#### Расстояние Джаро — Винклера
Расстояние Джаро — Винклера использует коэффициент масштабирования - p, что дает более благоприятные рейтинги строкам, которые совпадают друг с другом от начала до определённой длины - L, которая называется префиксом. Даны две строки s1 и s2. Их расстояние Джаро — Винклера dw это:
* dw = dj + (L * p * (1-dj));

где:
* dj — расстояние Джаро для строк s1 и s2
* L - длина общего префикса от начала строки до максимума 4-х символов. (цепочка совпавших символов с тождественными порядковыми номерами)
* p — постоянный коэффициент масштабирования, использующийся для того, чтобы скорректировать оценку в сторону повышения для выявления наличия общих префиксов. p не должен превышать 0,25, поскольку в противном случае расстояние может стать больше, чем 1. Стандартное значение этой константы в работе Винклера: p=0.1;

In [6]:
def jaro_vinkler_distance(str1,str2,p=0.1):
    if len(str1) > len(str2):
        s1 = str2
        s2 = str1
    else:
        s1 = str1
        s2 = str2
    dist = 0
    max_len = max(len(s1),len(s2))
    min_len = min(len(s1),len(s2))
    a = len(s1)
    b = len(s2)
    d = int(max_len/2)-1
    e = 0
    m = 0
    L = 0
    
    for i in range(min_len):
        if s1[i] == s2[i]:
            e += 1
            m += 1
            if i == L:
                L += 1
            continue 
        j = i-d
        if j<0 : 
            j=0
        if s2.find(s1[i],j,i+d) != -1:
            m += 1
    t = int((m - e + 1)/2) #добавлена +1 для корректногго округления
    if m==0:
        dj = 0
    else:
        dj = (m/a+m/b +(m-t)/m)/3
    if L>4: L=4
   
    
    return 1-(dj + (L * p * (1-dj)))
        

In [7]:
s3 ="РАЗРАБОТ"
s4 = "РАЗРАБОТКА"
jaro_distance(s3,s4)


0.06666666666666676

In [8]:
jaro_vinkler_distance(s3,s4)

0.040000000000000036

# Задание 4
Реализовать иерархическую кластеризацию.

Была реализована кластеризация агломеративным методом ближайшего соседа.

In [9]:
import numpy as np

In [10]:
def sortSecond(val): 
    return val[2]

In [11]:
def classification(text,dist_method = hemming_distance):
    
    tokens = tokinization(text)
    size = len(tokens)
    
    cluster = []
    y = []
    
    for i in range(size):
        y.append(i)
        for j in np.arange(i+1,size,1):
            dist = dist_method(tokens[i],tokens[j])
            cluster.append((i,j,dist))
    
    cluster.sort(key = sortSecond)
    
    num_cluster = len(y)
    for i in range(len(cluster)):
        pred_a = y[cluster[i][0]]
        pred_b = y[cluster[i][1]]
        if pred_a != pred_b:
            for j in range(len(y)):
                if (y[j] ==pred_a)|(y[j] ==pred_b):
                    y[j] = num_cluster
            y[cluster[i][0]] = y[cluster[i][1]] = num_cluster
            print("Step {}: clusters ({} and {})----> cluster: {} (dist={}) ".format(
                num_cluster-len(y)+1,pred_a,pred_b,num_cluster,cluster[i][2]))
            num_cluster += 1

In [12]:
test_text = "Мама руки раму"


In [13]:
classification(text,jaro_vinkler_distance)

Step 1: clusters (10 and 18)----> cluster: 30 (dist=0.11666666666666659) 
Step 2: clusters (24 and 28)----> cluster: 31 (dist=0.17500000000000004) 
Step 3: clusters (1 and 7)----> cluster: 32 (dist=0.20000000000000007) 
Step 4: clusters (32 and 20)----> cluster: 33 (dist=0.22499999999999998) 
Step 5: clusters (8 and 15)----> cluster: 34 (dist=0.30000000000000004) 
Step 6: clusters (34 and 29)----> cluster: 35 (dist=0.30000000000000004) 
Step 7: clusters (0 and 13)----> cluster: 36 (dist=0.32380952380952377) 
Step 8: clusters (2 and 35)----> cluster: 37 (dist=0.33333333333333337) 
Step 9: clusters (6 and 14)----> cluster: 38 (dist=0.33333333333333337) 
Step 10: clusters (30 and 17)----> cluster: 39 (dist=0.34444444444444444) 
Step 11: clusters (16 and 27)----> cluster: 40 (dist=0.35769230769230775) 
Step 12: clusters (39 and 36)----> cluster: 41 (dist=0.36111111111111116) 
Step 13: clusters (41 and 38)----> cluster: 42 (dist=0.375) 
Step 14: clusters (33 and 37)----> cluster: 43 (dist=0

In [14]:
classification(text,jaro_distance)

Step 1: clusters (10 and 18)----> cluster: 30 (dist=0.16666666666666663) 
Step 2: clusters (1 and 7)----> cluster: 31 (dist=0.22222222222222232) 
Step 3: clusters (31 and 20)----> cluster: 32 (dist=0.25) 
Step 4: clusters (24 and 28)----> cluster: 33 (dist=0.25) 
Step 5: clusters (2 and 15)----> cluster: 34 (dist=0.33333333333333337) 
Step 6: clusters (6 and 14)----> cluster: 35 (dist=0.33333333333333337) 
Step 7: clusters (8 and 34)----> cluster: 36 (dist=0.33333333333333337) 
Step 8: clusters (36 and 29)----> cluster: 37 (dist=0.33333333333333337) 
Step 9: clusters (30 and 17)----> cluster: 38 (dist=0.34444444444444444) 
Step 10: clusters (38 and 13)----> cluster: 39 (dist=0.36111111111111116) 
Step 11: clusters (32 and 37)----> cluster: 40 (dist=0.38888888888888895) 
Step 12: clusters (16 and 27)----> cluster: 41 (dist=0.39743589743589747) 
Step 13: clusters (0 and 9)----> cluster: 42 (dist=0.40476190476190477) 
Step 14: clusters (42 and 39)----> cluster: 43 (dist=0.4047619047619047

In [15]:
classification(text,hemming_distance)

Step 1: clusters (2 and 15)----> cluster: 30 (dist=0.5) 
Step 2: clusters (6 and 14)----> cluster: 31 (dist=0.5) 
Step 3: clusters (8 and 30)----> cluster: 32 (dist=0.5) 
Step 4: clusters (32 and 29)----> cluster: 33 (dist=0.5) 
Step 5: clusters (10 and 18)----> cluster: 34 (dist=0.5) 
Step 6: clusters (24 and 28)----> cluster: 35 (dist=0.5) 
Step 7: clusters (1 and 7)----> cluster: 36 (dist=0.6666666666666666) 
Step 8: clusters (36 and 33)----> cluster: 37 (dist=0.6666666666666666) 
Step 9: clusters (3 and 26)----> cluster: 38 (dist=0.6666666666666666) 
Step 10: clusters (0 and 9)----> cluster: 39 (dist=0.7142857142857143) 
Step 11: clusters (39 and 13)----> cluster: 40 (dist=0.7142857142857143) 
Step 12: clusters (40 and 21)----> cluster: 41 (dist=0.7142857142857143) 
Step 13: clusters (37 and 20)----> cluster: 42 (dist=0.75) 
Step 14: clusters (42 and 41)----> cluster: 43 (dist=0.75) 
Step 15: clusters (38 and 5)----> cluster: 44 (dist=0.75) 
Step 16: clusters (4 and 43)----> cluste